<a href="https://colab.research.google.com/github/vanderbilt-ml/ml-scikit-learn-demo/blob/main/sklearn-demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Machine Learning Using scikit-learn**

This notebook demonstrates the usage of pipelines to promote best practices for Machine Learning in Python. As a reminder, below are the following best practices we should follow when performing Machine Learning in Python:

1. Perform all pre-processing steps within cross-validation
2. Measure model performance (and model selection) using cross-validation
3. Follow literate programming practices to make code readable and make collaboration easier

## Problem Formulation

In this example, we will use Allison Horst's Palmer Penguins dataset, available here: https://github.com/allisonhorst/palmerpenguins.

The dataset contains data on 344 penguins. There are three different species of penguins in the dataset, collected from three separate islands in the Palmer Archipelago, Antarctica. 



## Load Libraries

First, we'll load our standard libraries for loading data, basic exploratory data analsysi (EDA), and machine learning. 

In [ ]:
#tables and visualizations


#machine learning


## Load Data

Here we first load the data into python using pandas and read it in as a pandas dataframe which is the format which we will use throughout the example. 

## Data cleaning and EDA

We can now explore our data. We leave this exercise to the reader. For now, we can observe that there are a few NA values which will likely need imputation. We'll wait for this step so that we can put it within our training loop. For now, we'll just drop all of the sex NAs out of the dataframe.

We'll take a quick look at the number of missing datapoints.

We want to predict the sex of the penguin. If the sex of the penguin is missing, we have no choice but to remove the observation. 

In [ ]:
penguins = penguins.dropna(subset=['sex'])
penguins.shape

(333, 8)

## Test-Train Splitting
Now, we'll set up sex as the classification column, set a random seed so we can reproduce our results, and set up our test-train split. 

Quick sanity check to make sure that everything seems correct for the train and test splits, first with the predictors, X

Then with the class, y

## Establishing the training pipeline

We can now establish the training pipeline for our models. Since this is a process we would need to repeat several times, it's good to essentially functionalize the process so we do not need to re-write redundant code. Here, we can impute some values that were missing, and encode any categorical values. Note that these pipelines will change according to the model and methodology you choose - additionally, the pipelines will also change depending on the data types of the columns in your dataset. 

In [ ]:
#individual pipelines for differing datatypes


In [ ]:
#establish preprocessing pipeline by columns


In [ ]:
#generate the whole modeling pipeline with preprocessing


#visualization for steps


Pipeline(steps=[('preproc',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat_pipe',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot_cat',
                                                                   OneHotEncoder(drop='if_binary'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f7c589d4280>),
                                                 ('num_pipe',
                                                  Pipeline(steps=[('impute_num',
                                                                   SimpleImputer()),
                                                                  ('scale_num',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f7c589d40d0>)])),
                ('mdl',
                 LogisticRegression(penalty='elasticnet', solver='saga',
                                    tol=0.01))])

## Cross-validation with Hyperparameter Tuning

Now that we have our pipelines, we can now use this as part of cross validation and hyperparameter tuning.

Let's look at the resulting tuning grid.

In [ ]:
tuning_grid

{'mdl__l1_ratio': array([0.  , 0.25, 0.5 , 0.75, 1.  ]),
 'mdl__C': array([1.00000000e-01, 3.16227766e+02, 1.00000000e+06])}

Now we'll actually do our model fitting and grid search. We'll only train on the training data, and leave the test data for when we have chosen our final model/parameters. 

This has fit multiple models. Which was the best?

And we can look at ALL of the cross-validation results as well:

In [ ]:
pd.DataFrame(grid_search.cv_results_)

## Best Model

The final fit here is already present in the generated model due to the way we set our parameters in the grid search. If we want to look at the performance, we can do so. Here is a non-helpful description of the best model:

In [ ]:
grid_search.best_estimator_

## Variable importance

Now we assess the importance in the selected model to reveal any potential insights.

In [ ]:
grid_search.classes_

In [ ]:
vip = grid_search.best_estimator_['mdl'].coef_[0]
vip

In [ ]:
#get names in correct preproc order
cat_names = grid_search.best_estimator_.named_steps['preproc'].transformers_[0][1].named_steps['onehot_cat'].get_feature_names()
num_names = grid_search.best_estimator_.named_steps['preproc'].transformers_[1][2]

#create df with vip info
coef_info = pd.DataFrame({'feat_names':np.hstack([cat_names, num_names]), 'vip': vip})

#get sign and magnitude information
coef_info = coef_info.assign(coef_mag = abs(coef_info['vip']),
                             coef_sign = np.sign(coef_info['vip']))

#sort and plot
coef_info = coef_info.set_index('feat_names').sort_values(by='coef_mag', ascending=False)
sns.barplot(y=coef_info.index, x='coef_mag', hue='coef_sign', data=coef_info, orient='h', dodge=False);

## Performance Metrics on Test Data


Here, we can see the performance of the model, which is pretty nice! We can also look into different scores specifically for more insight into the performance.

## Try it yourself!

Now that we've seen the power of pipelines in sklearn, let's now try implementing our own pipelines.

Try implementing a pipeline where we use median imputation for numeric columns instead of mean imputation, as well as the standard scaler. Implement the same imputers and encoders for categorical variables.

In [ ]:
#individual pipelines for differing datatypes
cat_pipeline = Pipeline(steps=[# your code here])
num_pipeline = Pipeline(steps=[# your code here])

#establish preprocessing pipeline by columns
preproc = ColumnTransformer([('cat_pipe', cat_pipeline, make_column_selector(dtype_include=object)),
                             ('num_pipe', num_pipeline, make_column_selector(dtype_include=np.number))],
                             remainder='passthrough')

With this new pipeline, now train a Random Forest model. Refer to the documentation for the parameters for the random forest classifier here: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [ ]:
#generate the whole modeling pipeline with preprocessing
pipe = Pipeline(steps=[('preproc', preproc),
                       ('mdl', # your code here)])

#visualization for steps
with config_context(display='diagram'):
    display(pipe)

Now perform cross validation and modify the n_estimators parameter to values of [100, 200,500] and max_depth parameter to values of [10,15,50] for the random forest classifier for hyperparameter tuning.

In [ ]:
tuning_grid = {'mdl__# your code here' :,
               'mdl__# your code here': }
grid_search = GridSearchCV(pipe, param_grid = tuning_grid, cv = 5, return_train_score=True)

In [ ]:
tuning_grid

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
print(grid_search.best_score_)
grid_search.best_params_

In [ ]:
pd.DataFrame(grid_search.cv_results_)

In [ ]:
grid_search.best_estimator_

In [ ]:
grid_search.classes_

Here, note that the process of getting the feature importances is slighly different than that for Logistic regression.

In [ ]:
vip = grid_search.best_estimator_['mdl'].feature_importances_
vip

In [ ]:
#get names in correct preproc order
cat_names = grid_search.best_estimator_.named_steps['preproc'].transformers_[0][1].named_steps['onehot_cat'].get_feature_names()
num_names = grid_search.best_estimator_.named_steps['preproc'].transformers_[1][2]

#create df with vip info
coef_info = pd.DataFrame({'feat_names':np.hstack([cat_names, num_names]), 'vip': vip})

#get sign and magnitude information
coef_info = coef_info.assign(coef_mag = abs(coef_info['vip']),
                             coef_sign = np.sign(coef_info['vip']))

#sort and plot
coef_info = coef_info.set_index('feat_names').sort_values(by='coef_mag', ascending=False)
sns.barplot(y=coef_info.index, x='coef_mag', hue='coef_sign', data=coef_info, orient='h', dodge=False);

In [ ]:
print(classification_report(y_test, grid_search.best_estimator_.predict(X_test)))

In [ ]:
cm = confusion_matrix(y_test, grid_search.best_estimator_.predict(X_test))
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                               display_labels=grid_search.classes_)
disp.plot()

plt.show()